In [1]:
import json
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import koreanize_matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import glob
# 데이터 전처리 후 시각화에 도움을 주는 라이브러리
import missingno as msno
# from xgboost import XGBClassifier
# from lightgbm import LGBMClassifier
from sklearn.impute import MissingIndicator, SimpleImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder, LabelEncoder
from sklearn import metrics
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, cross_val_score, KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import IsolationForest, RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report, confusion_matrix
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.pipeline import make_pipeline, Pipeline, FeatureUnion, make_union
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
import collections
import imblearn
from imblearn.over_sampling import ADASYN, RandomOverSampler, SMOTENC, SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN, SMOTETomek
import os
from bayes_opt import BayesianOptimization
from sklearn.base import TransformerMixin
from sklearn.base import BaseEstimator
from sklearn.decomposition import PCA

import tensorflow as tf
from scipy.stats import chi2_contingency, ttest_ind
import category_encoders as CE

# import eli5
# from eli5.sklearn import PermutationImportance

# for dirname, _, filenames in os.walk('../kaggle/MLproject_dataset'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

In [ ]:
# 1. 말뭉치 데이터셋을 준비합니다. texts는 입력 텍스트 데이터, labels는 해당 텍스트에 대한 레이블입니다.
# 2. 텍스트 데이터를 전처리합니다. Tokenizer를 사용하여 텍스트를 숫자 시퀀스로 변환하고, 패딩을 추가하여 시퀀스의 길이를 동일하게 맞춥니다.
# 3. 레이블 데이터를 전처리합니다. 레이블을 범주형 형식으로 변환합니다.
# 4. 모델을 구성합니다. Embedding 레이어로 단어 임베딩을 적용하고, LSTM 레이어를 사용하여 시퀀스 데이터를 처리합니다.
# 5. 모델을 컴파일합니다. 손실 함수와 옵티마이저를 지정합니다.
# 6. 모델을 훈련합니다. 입력 데이터와 레이블 데이터를 사용하여 모델을 학습시킵니다.
# 7. 예측을 수행합니다. 새로운 텍스트 데이터를 전처리한 후, 학습된 모델을 사용하여 예측값을 얻습니다.

import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 말뭉치 데이터셋 준비
texts = [...]  # 말뭉치 데이터셋 (입력 텍스트)
labels = [...]  # 말뭉치에 대한 레이블 (출력 예측값)

# 텍스트 데이터 전처리
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index
vocab_size = len(word_index) + 1
max_sequence_length = max([len(seq) for seq in sequences])
input_data = pad_sequences(sequences, maxlen=max_sequence_length)

# 레이블 데이터 전처리
label_data = tf.keras.utils.to_categorical(labels)

# 모델 구성
model = Sequential()
model.add(Embedding(vocab_size, 128, input_length=max_sequence_length))
model.add(LSTM(128))
model.add(Dense(len(labels), activation='softmax'))

# 모델 컴파일
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 모델 훈련
model.fit(input_data, label_data, epochs=10, batch_size=32)

# 예측하기
new_texts = [...]  # 예측할 텍스트 데이터
new_sequences = tokenizer.texts_to_sequences(new_texts)
new_data = pad_sequences(new_sequences, maxlen=max_sequence_length)
predictions = model.predict(new_data)

In [20]:
df = pd.read_csv('/Users/minsoo/Downloads/archive/Cleaned-Data.csv')
df

,Fever,Tiredness,Dry-Cough,Difficulty-in-Breathing,Sore-Throat,None_Sympton,Pains,Nasal-Congestion,Runny-Nose,Diarrhea,None_Experiencing,Age_0-9,Age_10-19,Age_20-24,Age_25-59,Age_60+,Gender_Female,Gender_Male,Gender_Transgender,Severity_Mild,Severity_Moderate,Severity_None,Severity_Severe,Contact_Dont-Know,Contact_No,Contact_Yes,Country
0,1,1,1,1,1,0,1,1,1,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,1,China
1,1,1,1,1,1,0,1,1,1,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,1,0,China
2,1,1,1,1,1,0,1,1,1,1,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0,0,China
3,1,1,1,1,1,0,1,1,1,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,China
4,1,1,1,1,1,0,1,1,1,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,China
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316795,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,1,0,Other
316796,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,1,1,0,0,Other
316797,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,Other
316798,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,Other


In [21]:
df.isnull().sum()

Fever                      0
Tiredness                  0
Dry-Cough                  0
Difficulty-in-Breathing    0
Sore-Throat                0
None_Sympton               0
Pains                      0
Nasal-Congestion           0
Runny-Nose                 0
Diarrhea                   0
None_Experiencing          0
Age_0-9                    0
Age_10-19                  0
Age_20-24                  0
Age_25-59                  0
Age_60+                    0
Gender_Female              0
Gender_Male                0
Gender_Transgender         0
Severity_Mild              0
Severity_Moderate          0
Severity_None              0
Severity_Severe            0
Contact_Dont-Know          0
Contact_No                 0
Contact_Yes                0
Country                    0
dtype: int64

In [ ]:

# 의료 전문서적 말뭉치 데이터셋이 여러 개의 txt 파일로 구성되어 있는 경우, 데이터셋을 효과적으로 학습시키기 위해 몇 가지 접근 방법을 고려할 수 있습니다. 
# 여기에는 데이터 로드, 전처리, 통합 등이 포함될 수 있습니다. 다음은 이러한 상황에서의 일반적인 절차를 안내해 드리겠습니다:

# 데이터 로드: 각각의 txt 파일을 개별적으로 로드합니다. 이를 위해 파일 경로를 지정하고 파일을 읽어 텍스트 데이터를 추출하는 방법을 사용할 수 있습니다.

# 데이터 전처리: 로드한 텍스트 데이터에 대해 필요한 전처리 단계를 수행합니다. 이는 텍스트 정제, 토큰화, 불용어 제거, 단어 임베딩 등을 포함할 수 있습니다. 
# 전처리 과정은 데이터의 특성과 목적에 따라 다르게 수행될 수 있습니다.

# 데이터 통합: 여러 개의 txt 파일로 구성된 데이터셋을 통합합니다. 이를 위해 로드하고 전처리한 데이터를 하나의 리스트로 결합합니다. 
# 예를 들어, 리스트로 각각의 txt 파일 데이터를 저장하고, 이들을 합쳐 전체 데이터셋을 구성할 수 있습니다.

# 모델 학습: 전처리 및 통합된 데이터셋을 사용하여 모델을 학습시킵니다. 적절한 딥러닝 모델 아키텍처를 선택하고, 입력 데이터와 출력 예측값을 사용하여 모델을 훈련시킬 수 있습니다.

# 예시 코드는 다음과 같이 구성될 수 있습니다:

import os

# 데이터 로드
data_directory = "data_directory"  # 데이터가 있는 디렉토리 경로
file_names = os.listdir(data_directory)
texts = []
labels = []

for file_name in file_names:
    file_path = os.path.join(data_directory, file_name)
    with open(file_path, 'r') as file:
        text = file.read()
        texts.append(text)
        labels.append(file_name)  # 필요에 따라 레이블을 지정할 수 있음

# 데이터 전처리
# ...

# 데이터 통합
combined_texts = texts
combined_labels = labels

# 모델 학습
# ...

In [5]:
# 위의 코드에서는 다음과 같은 단계를 거칩니다:

# 말뭉치 데이터셋을 준비합니다. texts는 입력 텍스트 데이터, labels는 해당 텍스트에 대한 레이블입니다.
# AutoTokenizer를 사용하여 원하는 토크나이저를 자동으로 로드합니다. 위 예시에서는 "bert-base-uncased" 모델의 토크나이저를 사용하도록 설정하였습니다.
# 텍스트 데이터를 전처리하고 토큰화합니다. tokenizer 객체를 사용하여 텍스트를 토큰화하고, 패딩과 자르기(truncation)를 적용하여 일관된 길이로 맞춥니다. return_tensors 매개변수를 통해 TensorFlow에서 사용할 수 있는 텐서 형태로 출력합니다.
# 레이블 데이터를 전처리합니다. 레이블을 범주형 형식으로 변환합니다.
# 모델 구성, 컴파일, 훈련은 이전 예시 코드와 동일하게 진행하면 됩니다.
# 예측을 수행합니다. 새로운 텍스트 데이터를 전처리하고 토큰화한 후, 학습된 모델을 사용하여 예측값을 얻습니다.

from transformers import AutoTokenizer

# 말뭉치 데이터셋 준비
# JSON 형식의 학습 데이터셋 로드
with open('/Users/minsoo/Downloads/154.의료, 법률 전문 서적 말뭉치/01.데이터/Training/02.라벨링데이터/Training_medical.json', 'r') as file:
    dataset = json.load(file)

# 학습 데이터셋 전처리
texts = []
labels = []

for data in dataset['data']:
    text = data['text']
    # label = data[['category', 'keyword']]
    category = data['category']
    keywords = data['keyword']
    label = f"{category} {', '.join(keywords)}"
    texts.append(text)
    labels.append(label)

# AutoTokenizer 로드
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# 텍스트 데이터 전처리 및 토큰화
input_data = tokenizer(texts, padding=True, truncation=True, max_length=128, return_tensors="tf")

# 레이블 데이터 전처리
label_data = tf.keras.utils.to_categorical(labels)

# 모델 구성, 컴파일, 훈련은 이전 예시 코드와 동일하게 진행하면 됩니다.

# 모델 구성
model = Sequential()
model.add(Embedding(vocab_size, 128, input_length=max_sequence_length))
model.add(LSTM(128))
model.add(Dense(len(labels), activation='softmax'))

# 모델 컴파일
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 모델 훈련
model.fit(input_data, label_data, epochs=10, batch_size=32)

# 예측하기
new_texts = [...]  # 예측할 텍스트 데이터

# 텍스트 데이터 전처리 및 토큰화
new_data = tokenizer(new_texts, padding=True, truncation=True, max_length=128, return_tensors="tf")

# 예측 수행
predictions = model.predict(new_data["input_ids"])


ValueError: invalid literal for int() with base 10: '재활의학/물리치료학/작업치료학 척추부, 근육, 받침대, 온열치료, 척추'